In [72]:
import numpy as np
import cudf
import cuspatial
from shapely.geometry import Point, Polygon
import geopandas
import time 



N = 10
lenpoly = 1000
N_polys = 40
polygon_list = [None]*N_polys
for j in range(N_polys):
    shift = np.random.uniform(-1,1)
    polygon = [[np.sin(x)+shift,np.cos(x)+shift] for x in np.linspace(0,2*np.pi,lenpoly)]
    polygon_arr = np.array(polygon)
    polygon_list[j] = polygon_arr

points = np.random.uniform(-1.5, 1.5, size=(N, 2))

st = time.time()
points_list = [Point(points[i,:]) for i in range(points.shape[0])]
shapekly_result_list = [None]*N_polys
for j in range(N_polys):
    shapely_poly = Polygon(polygon_list[j])
    shapely_result = shapely_poly.contains(points_list)
    shapekly_result_list[j] = shapely_result
end = time.time()

print(end-st)


st = time.time()

shapely_points_list = [Point(points[i,:]) for i in range(points.shape[0])]
shapely_poly_list = [Polygon(polygon) for polygon in polygon_list]

cupoints = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_points_list))
cupoly = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_poly_list))
current_index = 0
next_index = 0
while next_index < N_polys:
    if N_polys - current_index > 30:
        next_index = current_index + 30
    else:
        next_index = N_polys
    result = cuspatial.point_in_polygon(cupoints,
                                    cupoly[current_index:next_index]                   
                                    )
    current_index = next_index
end = time.time()

print(end-st)

st = time.time()
interleaved_points = points.flatten()
cupoints2 = cuspatial.GeoSeries.from_points_xy(interleaved_points)

shapely_poly = [Polygon(polygon) for polygon in polygon_list]
cupoly = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_poly))
print("length:",len(cupoly))


current_index = 0
next_index = 0
frames = []
while next_index < N_polys:
    if N_polys - current_index > 30:
        next_index = current_index + 30
    else:
        next_index = N_polys
    result = cuspatial.point_in_polygon(cupoints2,
                                    cupoly[current_index:next_index]                   
                                    )
    #df.update(result)
    current_index = next_index
    frames.append(result)
    
end = time.time()
print(end-st)

print(result)
result.loc[0]
result.values

df = cudf.concat(frames,axis=1)
print(df)
print(df.values[2,39])
type(df.values)

0.006578683853149414
0.15498900413513184
length: 40


/home/matthew/.local/share/virtualenvs/biopsylocalization-python-_12o-Huq/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/matthew/.local/share/virtualenvs/biopsylocalization-python-_12o-Huq/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/matthew/.local/share/virtualenvs/biopsylocalization-python-_12o-Huq/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/matthew/.local/share/virtualenvs/biopsylocalization-python-_12o-Huq/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 1 will likely res

0.1421678066253662
      30     31     32     33     34     35     36     37     38     39
0  False  False  False  False  False  False  False  False  False  False
1   True  False   True  False  False   True  False  False  False   True
2  False  False  False  False  False  False  False  False  False  False
3  False  False  False  False  False  False  False  False  False  False
4  False  False  False  False  False  False  False   True   True  False
5  False  False  False  False  False  False  False   True   True  False
6  False  False  False  False  False  False  False  False  False  False
7  False   True  False  False   True  False  False   True   True  False
8  False  False  False  False  False  False  False  False  False  False
9  False  False  False  False  False  False  False  False  False  False
      0      1      2      3      4      5      6      7      8      9   ...  \
0  False  False  False  False  False  False  False  False  False  False  ...   
1  False  False  False  False

cupy.ndarray

In [88]:
import numpy as np
import cuspatial
from shapely.geometry import Point, Polygon

N=10

points = np.random.uniform(-1.5, 1.5, size=(N, 2))
shapely_points_list = [Point(points[i,:]) for i in range(points.shape[0])]
cupoints = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_points_list))
print(cupoints)
print(cupoints[0:10])

cupoints.points


0     POINT (-1.39248 0.60009)
1     POINT (1.06403 -1.13638)
2    POINT (-0.54963 -0.82837)
3    POINT (-1.47684 -0.88845)
4     POINT (0.19736 -0.91442)
5     POINT (-1.39196 1.09605)
6     POINT (-0.87897 1.31730)
7      POINT (0.22517 0.06904)
8      POINT (1.44191 1.12307)
9     POINT (-0.89762 0.14314)
dtype: geometry
0     POINT (-1.39248 0.60009)
1     POINT (1.06403 -1.13638)
2    POINT (-0.54963 -0.82837)
3    POINT (-1.47684 -0.88845)
4     POINT (0.19736 -0.91442)
5     POINT (-1.39196 1.09605)
6     POINT (-0.87897 1.31730)
7      POINT (0.22517 0.06904)
8      POINT (1.44191 1.12307)
9     POINT (-0.89762 0.14314)
dtype: geometry


In [26]:
import numpy as np

a = np.array([[[1,1,1],[2,2,2],[3,3,3],[4,4,4]],[[1,1,1],[2,2,2],[3,3,3],[4,4,4]],[[5,1,1],[2,2,2],[3,3,3],[4,6,4]]])
a.shape[1]

a[0:2]

b = np.reshape(a,(-1,3))
print(b)
print(b[:,2])

[[1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]
 [1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]
 [5 1 1]
 [2 2 2]
 [3 3 3]
 [4 6 4]]
[1 2 3 4 1 2 3 4 1 2 3 4]


In [41]:
from bisect import bisect_left


def take_closest(myList_org, myNumber):
    """
    Assumes myList is sorted. Returns index of closest value and the closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    myList = myList_org.copy()
    if myList[0] > myList[1]:
        myList.reverse()
        list_reversed = True
    else:
        list_reversed = False
    if list_reversed == False:
        pos = bisect_left(myList, myNumber)
        if pos == 0:
            return 0, myList[0]
        if pos == len(myList):
            return pos-1, myList[-1]
        before = myList[pos - 1]
        after = myList[pos]
        if after - myNumber < myNumber - before:
            return pos, after
        else:
            return pos - 1, before
    elif list_reversed == True:
        pos = bisect_left(myList, myNumber)
        if pos == 0:
            return len(myList)-1, myList[0]
        if pos == len(myList):
            return 0, myList[-1]
        before = myList[pos - 1]
        after = myList[pos]
        if after - myNumber < myNumber - before:
            return len(myList) - 1 - pos, after
        else:
            return len(myList) - pos, before


def take_closest_array_input(myList, myArray):
    """
    Assumes myList is sorted. Returns index of closest value and the closest value to myNumber.

    If two numbers are equally close, return the smallest number. Takes an array as input and vectorizes the 'take_closest_index_only' function. 

    Returns an array of the indices of the original list that contains the value that is closest to each element of the given array.
    """

    take_closest_index_only_vectorized = np.vectorize(take_closest, excluded=['myList_org'])
    closest_vals_indices_array, closest_vals_array = take_closest_index_only_vectorized(myList_org = myList,myNumber = myArray)
    return closest_vals_indices_array, closest_vals_array


a = [1,3,5,7,8,9,10,12,14,15,16,17,18,19]

b = np.array([3.4,5.6,4.8,8.8,1.5,6.3,6.4,10.7,19,13])

c, d = take_closest_array_input(a,b)

print(c)
print(d)



[ 1  2  2  5  0  3  3  6 13  7]
[ 3  5  5  9  1  7  7 10 19 12]


In [86]:
import cupy as cp

a = [cp.array(True) for j in range(10)]

b = cp.array(a)

df = cudf.DataFrame.from_dict({"a":a})
b

color = cp.array([1,0,0],dtype=float)

color

array([1., 0., 0.])

In [83]:
a = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

for j,i in enumerate(a):
    print(j % 10)


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


In [94]:
import cudf
import numpy as np

N=10

points = [np.array([1,1,1]),np.array([1,2,1]),np.array([1,1,2])]

dictionary = {"points": points}

a = cudf.DataFrame.from_dict(dictionary)
a

,points
0,"[1, 1, 1]"
1,"[1, 2, 1]"
2,"[1, 1, 2]"


In [95]:
[int(i) for i in range(5) for j in range(3)]

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]

In [4]:
import numpy as np

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b=np.array([0,1])
c=np.array([1,2])
c = a[b,c]
c

array([2, 6])

In [6]:
import numpy as np

def color_by_bool_by_arr(contain_bool_arr):
    color_by_bool_vectorized = np.vectorize(color_by_bool)
    color_arr = color_by_bool_vectorized(contain_bool_arr)
    return color_arr



def color_by_bool(bool_val):
    if bool_val == True:
        color = np.array([0,1,0],dtype=float)
    elif bool_val == False:
        color = np.array([1,0,0],dtype=float)

    return color

a = np.array([True, False, True])

for element in a:
    print(element)

True
False
True


In [12]:
import cupy as cp 
import time  
import numpy as np


def color_by_bool_by_arr(contain_bool_arr):
    color_list = [None]*len(contain_bool_arr)
    for index,element_bool in enumerate(contain_bool_arr):
        if element_bool == True:
            color = np.array([0,1,0])
        else:
            color = np.array([1,0,0])
        color_list[index] = color
    return color_list

a = cp.random.randint(2, size=10000)
st = time.time()
b= color_by_bool_by_arr(a)

end = time.time()
print(end-st)
a = np.random.randint(2, size=10000)


st = time.time()
b= color_by_bool_by_arr(a)

end = time.time()
print(end-st)


1.077784538269043
0.013470649719238281


In [11]:
import cudf
import pandas
import numpy as np

example_dict = {"1":[True, False, True],"2":[False, False, True]}

df = cudf.DataFrame(example_dict)

print(df)

df.to_numpy()[[0,1,2],[1,1,1]]







       1      2
0   True  False
1  False  False
2   True   True


array([False, False,  True])

In [50]:
import numpy as np

a = np.empty([10,3])
a[2,:] = np.array([1,1,1])
a

array([[4.66881057e-310, 0.00000000e+000, 4.78104785e+180],
       [3.94811624e+180, 7.29010818e+045, 2.55388173e+151],
       [1.00000000e+000, 1.00000000e+000, 1.00000000e+000],
       [6.53042243e+015, 2.00291244e-076, 2.01843055e-076],
       [2.10479226e-076, 2.19115395e-076, 1.18027089e+180],
       [8.90643897e+179, 1.70544946e+214, 1.36936617e+069],
       [8.79819426e+252, 7.01219261e+194, 2.28221981e+243],
       [5.45385516e+069, 9.75023642e+199, 1.74552498e-076],
       [5.14283944e+025, 5.12318582e+025, 4.00234504e+035],
       [5.23279660e+261, 3.02253513e-066, 5.17130405e-310]])